<h2>Importing Packages</h2>

In [4]:

import numpy as np
import pandas as pd

<h2>Importing Datasets</h2>

In [5]:
Tweet_Train = pd.DataFrame(pd.read_csv('twitter_training.csv'))
Tweet_Test = pd.DataFrame(pd.read_csv('twitter_validation.csv'))

<h2>Understanding the Data</h2>

In [6]:
headers=['Tweet ID', 'Entity', 'Sentiment', 'Content']
Tweet_Train.columns = headers
Tweet_Test.columns = headers
Tweet_Train.head(25)

,Tweet ID,Entity,Sentiment,Content
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
5,2402,Borderlands,Positive,So I spent a few hours making something for fu...
6,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
7,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
8,2402,Borderlands,Positive,So I spent a few hours making something for fu...
9,2402,Borderlands,Positive,2010 So I spent a few hours making something f...


In [7]:
Tweet_Train.tail(25)

,Tweet ID,Entity,Sentiment,Content
74656,9196,Nvidia,Negative,really doesn't look bad btw!.. net.com/google-...
74657,9197,Nvidia,Neutral,Nvidia doesn’t want to give up its 2017 ‘crypt...
74658,9197,Nvidia,Neutral,"Nvidia plans to release its 2017 ""Crypto Craze..."
74659,9197,Nvidia,Neutral,"Nvidia does not want to give up its ""cryptoins..."
74660,9197,Nvidia,Neutral,Nvidia doesn’t intend to give away its 2017 ad...
74661,9197,Nvidia,Neutral,Nvidia therefore doesn ’ t want to give up its...
74662,9197,Nvidia,Neutral,is doesn’t should I give up its password ‘cryp...
74663,9198,Nvidia,Negative,Nvidia really delayed the 3070 2 weeks .
74664,9198,Nvidia,Negative,Nvidia really delayed the 3070 by 2 weeks.
74665,9198,Nvidia,Negative,Nvidia did delay by 3070 2 weeks.
